In [ ]:
import pexpect

# Function to configure a router via Telnet
def configure_router_via_telnet(host, port, commands):
    try:
        # Connect to the router
        connection = pexpect.spawn(f"telnet {host} {port}", encoding="utf-8")
        
        # Wait for the initial prompt
        connection.expect("Router>")
        
        # Enter enable mode
        connection.sendline("enable")
        connection.expect("Router#")
        
        # Enter configuration mode
        connection.sendline("configure terminal")
        connection.expect("(config)#")
        
        # Send commands
        for command in commands:
            connection.sendline(command)
            connection.expect("(config.*)#")
        
        # Save the configuration
        connection.sendline("write memory")
        connection.expect("Router#")
        
        # Exit configuration mode
        connection.sendline("exit")
        connection.expect("Router#")
        
        # Close the connection
        connection.sendline("exit")
        print(f"Configuration applied to router at {host}:{port}")
    
    except pexpect.exceptions.EOF:
        print(f"Connection to {host}:{port} was closed unexpectedly.")
    except pexpect.exceptions.TIMEOUT:
        print(f"Connection to {host}:{port} timed out.")

# Example usage
if __name__ == "__main__":
    host = "127.0.0.1"
    ports = [5000, 5001, 5002, 5003, 5004, 5005, 5006]  # Telnet ports for 7 routers
    commands = [
        "interface g1/0",
        "ipv6 address 2001:db8:1::1/64",
        "ipv6 enable",
        "exit",
        "ipv6 router rip GNS3",
        "interface g1/0",
        "ipv6 rip GNS3 enable",
        "exit",
    ]

    for port in ports:
        configure_router_via_telnet(host, port, commands)


In [ ]:
import pexpect
print(dir(pexpect))

In [ ]:
from Exscript.protocols import Telnet

# Define router details
host = "127.0.0.1"
port = 5000  # Telnet port for the router
username = "cisco"
password = "cisco"

# Commands to configure the router
commands = [
    "enable",
    "configure terminal",
    "interface g1/0",
    "ipv6 address 2001:db8:1::1/64",
    "ipv6 enable",
    "exit",
    "ipv6 router rip GNS3",
    "interface g1/0",
    "ipv6 rip GNS3 enable",
    "exit",
    "write memory",
]

# Connect to the router
conn = Telnet()
conn.connect(host, port=port)
#conn.login(username, password)

# Send commands
for command in commands:
    print(command)
    conn.execute(command)
    print(command)

# Close the connection
conn.close()
print("Configuration applied successfully.")

In [11]:
import json
import ipaddress
from Exscript.protocols import Telnet

def load_json(filename):
    with open(filename, 'r') as file:
        return json.load(file)

def get_connections(routeur_data):
    connections = []
    erreur = []

    for routeur, config in routeur_data.items():
        for interface, voisin in config.get('interface', {}).items():
            for routeur_voisin, interface_voisin in voisin.items():
                connection = tuple(sorted([(routeur, interface), (routeur_voisin, interface_voisin)]))
                if connection not in connections:
                    connections.append(connection)

                config_voisin = routeur_data.get(routeur_voisin, {}).get('interface', {}).get(interface_voisin, {})
                if config_voisin and config_voisin.get(routeur) != interface:
                    erreur.append(f"Incohérence: {routeur} ({interface}) <-> {routeur_voisin} ({interface_voisin})")

    return connections, erreur

def get_subnets_and_router_ips(connections, routeur_data, as_data):
    subnets = {}
    router_ips = {}
    as_subnets = {}
    subnet_routers = {}

    # Initialisation des sous-réseaux par AS
    for as_id, as_entry in as_data.items():
        subnets_brut = as_entry.get('plage_adresse', [])
        as_subnets[as_id] = []
        for subnet_brut in subnets_brut:
            network = ipaddress.IPv6Network(subnet_brut)
            as_subnets[as_id].extend(network.subnets(new_prefix=64))
        as_subnets[as_id] = iter(as_subnets[as_id])

    tous_les_subnets = set()

    # Attribution des sous-réseaux et des adresses IP
    for (routeur1, interface1), (routeur2, interface2) in connections:
        if (routeur1, interface1) not in subnets and (routeur2, interface2) not in subnets:
            as_id1 = str(routeur_data[routeur1]['AS_number'])
            as_id2 = str(routeur_data[routeur2]['AS_number'])
            as_id = as_id1 if as_id1 <= as_id2 else as_id2
            
            subnet = next(as_subnets[as_id])
            while subnet in tous_les_subnets:
                subnet = next(as_subnets[as_id])
            
            subnets[(routeur1, interface1)] = str(subnet)
            subnets[(routeur2, interface2)] = str(subnet)
            tous_les_subnets.add(subnet)
            subnet_routers[subnet] = set([(routeur1,interface1), (routeur2,interface2)])
        else:
            if (routeur1, interface1) in subnets:
                subnet = ipaddress.IPv6Network(subnets[(routeur1, interface1)])
                subnets[(routeur2, interface2)] = str(subnet)
            else:
                subnet = ipaddress.IPv6Network(subnets[(routeur2, interface2)])
                subnets[(routeur1, interface1)] = str(subnet)
            
            subnet_routers[subnet].add((routeur1,interface1))
            subnet_routers[subnet].add((routeur2,interface2))

    # Attribution des adresses IP aux routeurs
    for subnet, routers in subnet_routers.items():
        for i, router in enumerate(routers):
            if router not in router_ips:
                router_ips[router] = str(subnet[i+1])

    return subnets, router_ips

def affiche_connexion(connections, subnets):
    print("Connexions entre routeurs avec sous-réseaux:")
    for (routeur1, interface1), (routeur2, interface2) in connections:
        subnet1 = subnets.get((routeur1, interface1), "Non attribué")
        print(f"{routeur1} ({interface1}) <-> {routeur2} ({interface2}): {subnet1}")

def affiche_erreur(erreurs):
    if erreurs:
        print("\nIncohérences détectées:")
        for erreur in erreurs:
            print(erreur)
    else:
        print("\nAucune incohérence trouvée.")

def configure_routeur_telnet(routeur, config, subnets, ips, connections):
    """Configure les routeurs via Telnet avec Exscript."""
    try:
        host = "localhost"
        port = config['port_telnet']
        conn = Telnet()
        conn.connect(host, port)
        print(f"Connexion à {routeur} sur le port {port}...")

        conn.send("\rconfigure terminal\r")
        conn.send("ipv6 unicast-routing\r")
        for (r, interface), subnet in subnets.items():
            if r == routeur and subnet != "Aucune plage disponible":
                ipv6_address = ips[(r,interface)]
                conn.send(f"interface {interface}\r")
                conn.send(f"ipv6 address {ipv6_address}/{ipaddress.IPv6Network(subnet).prefixlen}\r")
                conn.send(f"ipv6 enable\r")
                conn.send("no shutdown\r")
                conn.send("exit\r")
        conn.send("end\r")
        conn.send("exit\r")
        print(f"Configuration de {routeur} terminée.")

    except Exception as e:
        print(f"Erreur lors de la configuration de {routeur}: {e}")

if __name__ == "__main__":

    as_data = load_json("as.json")
    routeur_data = load_json("router.json")

    connections, erreurs = get_connections(routeur_data)

    subnets, ips = get_subnets_and_router_ips(connections, routeur_data, as_data)
    affiche_connexion(connections, subnets)
    affiche_erreur(erreurs)

    for routeur, config in routeur_data.items():
        configure_routeur_telnet(routeur, config, subnets, ips, connections)


KeyboardInterrupt: 

In [9]:
arf = load_json("as.json")
print(arf)

rty = load_json("router.json")
print(rty)

{'1': {'plage_adresse': ['2001:db8::/32']}, '2': {'plage_adresse': ['2001:db9::/32']}}
{'R1': {'AS_number': 1, 'port_telnet': 5000, 'interface': {'g1/0': {'R2': 'g1/0'}}}, 'R2': {'AS_number': 2, 'port_telnet': 5001, 'interface': {'g1/0': {'R1': 'g1/0'}}}}


In [13]:
import Exscript.protocols
print("Exscript is installed and working!")

Exscript is installed and working!


In [12]:
import json
import ipaddress
from Exscript.protocols import Telnet

def load_json(filename):
    print(f"Loading JSON data from {filename}...")
    with open(filename, 'r') as file:
        data = json.load(file)
    print(f"Loaded {len(data)} entries from {filename}.")
    return data

def get_connections(routeur_data):
    print("Generating connections and checking for inconsistencies...")
    connections = []
    erreur = []

    for routeur, config in routeur_data.items():
        print(f"Processing router {routeur}...")
        for interface, voisin in config.get('interface', {}).items():
            for routeur_voisin, interface_voisin in voisin.items():
                connection = tuple(sorted([(routeur, interface), (routeur_voisin, interface_voisin)]))
                if connection not in connections:
                    connections.append(connection)

                config_voisin = routeur_data.get(routeur_voisin, {}).get('interface', {}).get(interface_voisin, {})
                if config_voisin and config_voisin.get(routeur) != interface:
                    erreur.append(f"Incohérence: {routeur} ({interface}) <-> {routeur_voisin} ({interface_voisin})")
    print(f"Generated {len(connections)} connections.")
    return connections, erreur

def get_subnets_and_router_ips(connections, routeur_data, as_data):
    print("Allocating subnets and router IPs...")
    subnets = {}
    router_ips = {}
    as_subnets = {}
    subnet_routers = {}

    # Initializing subnets for each AS
    for as_id, as_entry in as_data.items():
        print(f"Processing AS {as_id}...")
        subnets_brut = as_entry.get('plage_adresse', [])
        as_subnets[as_id] = []
        for subnet_brut in subnets_brut:
            network = ipaddress.IPv6Network(subnet_brut)
            as_subnets[as_id].extend(network.subnets(new_prefix=64))
        as_subnets[as_id] = iter(as_subnets[as_id])

    tous_les_subnets = set()

    # Assign subnets and router IPs
    for (routeur1, interface1), (routeur2, interface2) in connections:
        print(f"Processing connection between {routeur1} and {routeur2}...")
        if (routeur1, interface1) not in subnets and (routeur2, interface2) not in subnets:
            as_id1 = str(routeur_data[routeur1]['AS_number'])
            as_id2 = str(routeur_data[routeur2]['AS_number'])
            as_id = as_id1 if as_id1 <= as_id2 else as_id2
            
            subnet = next(as_subnets[as_id])
            while subnet in tous_les_subnets:
                subnet = next(as_subnets[as_id])
            
            subnets[(routeur1, interface1)] = str(subnet)
            subnets[(routeur2, interface2)] = str(subnet)
            tous_les_subnets.add(subnet)
            subnet_routers[subnet] = set([(routeur1, interface1), (routeur2, interface2)])
        else:
            if (routeur1, interface1) in subnets:
                subnet = ipaddress.IPv6Network(subnets[(routeur1, interface1)])
                subnets[(routeur2, interface2)] = str(subnet)
            else:
                subnet = ipaddress.IPv6Network(subnets[(routeur2, interface2)])
                subnets[(routeur1, interface1)] = str(subnet)
            
            subnet_routers[subnet].add((routeur1, interface1))
            subnet_routers[subnet].add((routeur2, interface2))

    # Assign IPs to routers
    for subnet, routers in subnet_routers.items():
        print(f"Assigning IPs in subnet {subnet}...")
        for i, router in enumerate(routers):
            if router not in router_ips:
                router_ips[router] = str(subnet[i+1])

    print("Subnet and router IP allocation complete.")
    return subnets, router_ips

def affiche_connexion(connections, subnets):
    print("\nConnections between routers with subnets:")
    for (routeur1, interface1), (routeur2, interface2) in connections:
        subnet1 = subnets.get((routeur1, interface1), "Non attribué")
        print(f"{routeur1} ({interface1}) <-> {routeur2} ({interface2}): {subnet1}")

def affiche_erreur(erreurs):
    if erreurs:
        print("\nInconsistencies detected:")
        for erreur in erreurs:
            print(erreur)
    else:
        print("\nNo inconsistencies found.")

def configure_routeur_telnet(routeur, config, subnets, ips, connections):
    print(f"Configuring router {routeur} via Telnet...")
    try:
        host = "localhost"
        port = config['port_telnet']
        conn = Telnet()
        conn.connect(host, port)
        print(f"Connected to {routeur} on port {port}.")

        conn.send("\rconfigure terminal\r")
        conn.send("ipv6 unicast-routing\r")
        for (r, interface), subnet in subnets.items():
            if r == routeur and subnet != "Aucune plage disponible":
                ipv6_address = ips[(r, interface)]
                conn.send(f"interface {interface}\r")
                conn.send(f"ipv6 address {ipv6_address}/{ipaddress.IPv6Network(subnet).prefixlen}\r")
                conn.send("ipv6 enable\r")
                conn.send("no shutdown\r")
                conn.send("exit\r")
        conn.send("end\r")
        conn.send("exit\r")
        print(f"Configuration of {routeur} completed.")

    except Exception as e:
        print(f"Error configuring {routeur}: {e}")

if __name__ == "__main__":
    print("Loading JSON files...")
    as_data = load_json("as.json")
    routeur_data = load_json("router.json")

    print("Generating connections and subnets...")
    connections, erreurs = get_connections(routeur_data)
    subnets, ips = get_subnets_and_router_ips(connections, routeur_data, as_data)

    print("Displaying connections and errors...")
    affiche_connexion(connections, subnets)
    affiche_erreur(erreurs)

    print("\nStarting router configurations...")
    for routeur, config in routeur_data.items():
        configure_routeur_telnet(routeur, config, subnets, ips, connections)
    print("\nRouter configuration completed.")


Loading JSON files...
Loading JSON data from as.json...
Loaded 2 entries from as.json.
Loading JSON data from router.json...
Loaded 2 entries from router.json.
Generating connections and subnets...
Generating connections and checking for inconsistencies...
Processing router R1...
Processing router R2...
Generated 1 connections.
Allocating subnets and router IPs...
Processing AS 1...


KeyboardInterrupt: 